# Abundance Maps

In [1]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import yaml

In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse

In [3]:
figure_setup()
settings = Settings()

In [4]:
CONFIG_FILE = "02"

In [5]:
config = yaml.safe_load(open(f"../configs/{CONFIG_FILE}.yml"))

In [6]:
def read_data(simulation: str, abundances: list) -> pd.DataFrame:
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=config["DISC_STD_CIRC"],
        disc_min_circ=config["DISC_MIN_CIRC"],
        cold_disc_delta_circ=config["COLD_DISC_DELTA_CIRC"],
        bulge_max_specific_energy=config["BULGE_MAX_SPECIFIC_ENERGY"])
    for abundance in abundances:
        s.add_metal_abundance(of=abundance[0], to=abundance[1])

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {"ComponentTag": s.region_tag[is_real_star & is_main_obj]}
    for abundance in abundances:
        of, to = abundance
        props[f"[{of}/{to}]"] = s.metal_abundance[f"{of}/{to}"][
            is_real_star & is_main_obj]

    df = pd.DataFrame(props)
    df[~np.isfinite(df)] = np.nan
    df.dropna(inplace=True)

    return df

In [7]:
def plot_map_for_simulation(simulation: str, abundances: list,
                            xlim: tuple, xticks: list,
                            ylim: tuple, yticks: list) -> None:
    settings = Settings()
    galaxy = parse(simulation)[0]
    label = f"Au{galaxy}"
    df = read_data(simulation=simulation, abundances=abundances)

    x_label = f"[{abundances[0][0]}/{abundances[0][1]}]"
    y_label = f"[{abundances[1][0]}/{abundances[1][1]}]"

    fig = plt.figure(figsize=(3.0, 2.5))
    gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
    ax = gs.subplots(sharex=True, sharey=True)

    ax.set_xlim(xlim)
    ax.set_xticks(xticks)
    ax.set_xlabel(x_label)

    ax.set_ylim(ylim)
    ax.set_yticks(yticks)
    ax.set_ylabel(y_label)

    ax.tick_params(which='both', direction="in")
    ax.label_outer()

    im = ax.hist2d(
            df[x_label], df[y_label],
            cmap='nipy_spectral', bins=config["ABUNDANCE_MAPS"]["N_BINS"],
            range=[ax.get_xlim(), ax.get_ylim()],
            norm=mcolors.LogNorm(
                vmin=config["ABUNDANCE_MAPS"]["MIN_PART_COUNT"],
                vmax=config["ABUNDANCE_MAPS"]["MAX_PART_COUNT"]),
            rasterized=True)[-1]

    for component in settings.components:
        is_region = (df["ComponentTag"] == settings.component_tags[component])

        median_x = np.nanmedian(df[x_label][is_region])
        median_y = np.nanmedian(df[y_label][is_region])

        color = settings.component_colors[component]
        marker = settings.component_markers[component]
        comp_label = settings.component_labels[component]
        ax.scatter(median_x, median_y, c=color, s=30, lw=1, edgecolors='k',
                   zorder=15, marker=marker, label=comp_label)

    cbar = fig.colorbar(im, ax=ax, orientation='vertical',
                        label=r'$N_\star$', pad=0)
    cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

    ax.legend(loc="lower left", framealpha=0.75, fontsize=5.0,
              scatterpoints=1, edgecolor="None", fancybox=True)

    ax.text(x=ax.get_xlim()[0], y=ax.get_ylim()[1],
            s=r"$\texttt{" + label + "}$", size=8.0, ha='left', va='bottom')

    fig.savefig(
        f"../images/metallicity/"
        f"map_OFe_vs_FeH_{simulation}{config['FILE_SUFFIX']}.pdf")
    plt.close(fig)

In [20]:
def plot_map_by_component_for_simulation(
        simulation: str, abundances: list,
        xlim: tuple, xticks: list,
        ylim: tuple, yticks: list) -> None:
    settings = Settings()
    galaxy = parse(simulation)[0]
    label = f"Au{galaxy}"
    df = read_data(simulation=simulation, abundances=abundances)

    x_label = f"[{abundances[0][0]}/{abundances[0][1]}]"
    y_label = f"[{abundances[1][0]}/{abundances[1][1]}]"

    fig = plt.figure(figsize=(8.0, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs:
        ax.set_xlim(xlim)
        ax.set_xticks(xticks)
        ax.set_xlabel(x_label)

        ax.set_ylim(ylim)
        ax.set_yticks(yticks)
        ax.set_ylabel(y_label)

        ax.tick_params(which='both', direction="in")
        ax.label_outer()

    im = axs[0].hist2d(
            df[x_label], df[y_label],
            cmap='nipy_spectral', bins=config["ABUNDANCE_MAPS"]["N_BINS"],
            range=[ax.get_xlim(), ax.get_ylim()],
            norm=mcolors.LogNorm(
                vmin=config["ABUNDANCE_MAPS"]["MIN_PART_COUNT"],
                vmax=config["ABUNDANCE_MAPS"]["MAX_PART_COUNT"]),
            rasterized=True)[-1]
    axs[0].text(
        x=0.05, y=0.05, size=8.0, ha="left", va="bottom",
        s=r"$\textbf{All}$", c="black", transform=axs[0].transAxes)

    for i, c in enumerate(settings.components):
        ax = axs[i + 1]
        is_region = (df["ComponentTag"] == settings.component_tags[c])
        im = ax.hist2d(
            df[x_label][is_region],
            df[y_label][is_region],
            cmap='nipy_spectral', bins=config["ABUNDANCE_MAPS"]["N_BINS"],
            range=[ax.get_xlim(), ax.get_ylim()],
            norm=mcolors.LogNorm(
                vmin=config["ABUNDANCE_MAPS"]["MIN_PART_COUNT"],
                vmax=config["ABUNDANCE_MAPS"]["MAX_PART_COUNT"]),
            rasterized=True)[-1]
        ax.text(
            x=0.05, y=0.05, size=8.0, ha="left", va="bottom",
            s=r"$\textbf{" + list(settings.component_labels.values())[i] \
                + "}$",
            c=list(settings.component_colors.values())[i],
            transform=ax.transAxes)
        

        median_x = np.nanmedian(df[x_label][is_region])
        median_y = np.nanmedian(df[y_label][is_region])

        color = settings.component_colors[c]
        marker = settings.component_markers[c]
        ax.scatter(
            np.nanmedian(df[x_label][is_region]),
            np.nanmedian(df[y_label][is_region]),
            c=color, s=30, lw=1, edgecolors='k',
            zorder=15, marker=marker)

    cbar = fig.colorbar(im, ax=axs[-1], orientation='vertical',
                        label=r'$N_\star$', pad=0)
    cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

    axs[0].text(
        x=0.05, y=0.95,
        s=r"$\texttt{" + label + "}$", size=8.0, ha='left', va='top',
        transform=axs[0].transAxes)

    fig.savefig(
        f"../images/metallicity/"
        f"map_OFe_vs_FeH_by_component_{simulation}{config['FILE_SUFFIX']}.pdf")
    plt.close(fig)

In [21]:
simulation = "au6_or_l4_s127"
plot_map_by_component_for_simulation(
    simulation=simulation,
    abundances=[("Fe", 'H'), ('O', "Fe")],
    xlim=(-1, 1), xticks=[-0.5, 0, 0.5],
    ylim=(0.15, 0.35), yticks=[0.2, 0.25, 0.3])

In [32]:
def plot_map_for_sample(sample: list, abundances: list,
                        xlim: tuple, xticks: list,
                        ylim: tuple, yticks: list) -> None:

    x_label = f"[{abundances[0][0]}/{abundances[0][1]}]"
    y_label = f"[{abundances[1][0]}/{abundances[1][1]}]"

    fig = plt.figure(figsize=(7, 8))
    gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flatten():
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(xlim)
        ax.set_xticks(xticks)
        ax.set_ylim(ylim)
        ax.set_yticks(yticks)
        ax.set_axisbelow(True)
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel(x_label)
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(y_label)
        if ax == axs[-1, -1]:  # Draw symbol legend
            for i, c in enumerate(settings.components):
                color = settings.component_colors[c]
                marker = settings.component_markers[c]
                ax.scatter(0.1, 0.7 - 0.15 * (i + 1), c=color, s=30, lw=1,
                           marker=marker, ec='k', zorder=15,
                           transform=ax.transAxes)
                ax.text(x=0.2, y=0.7 - 0.15 * (i + 1), size=8.0, c=color,
                        ha="left", va="center", transform=ax.transAxes,
                        s=r"$\textbf{" \
                            + list(settings.component_labels.values())[i] \
                            + "}$",)

    for i, simulation in enumerate(sample):
        ax = axs.flatten()[i]
        galaxy = parse(simulation)[0]
        label = f"Au{galaxy}"
        df = read_data(simulation=simulation, abundances=abundances)

        ax.hist2d(
            df[x_label], df[y_label],
            cmap='nipy_spectral', bins=config["ABUNDANCE_MAPS"]["N_BINS"],
            range=[ax.get_xlim(), ax.get_ylim()],
            norm=mcolors.LogNorm(
                vmin=config["ABUNDANCE_MAPS"]["MIN_PART_COUNT"],
                vmax=config["ABUNDANCE_MAPS"]["MAX_PART_COUNT"]),
            rasterized=True)[-1]

        for c in settings.components:
            is_region = (
                df["ComponentTag"] == settings.component_tags[c])

            median_x = np.nanmedian(df[x_label][is_region])
            median_y = np.nanmedian(df[y_label][is_region])

            color = settings.component_colors[c]
            marker = settings.component_markers[c]
            comp_label = settings.component_labels[c]
            ax.scatter(median_x, median_y, c=color, s=30, lw=1, edgecolors='k',
                       zorder=15, marker=marker)

        ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
                size=6.0, transform=ax.transAxes, ha='left', va='top')

        fig.savefig(
            f"../images/metallicity/"
            f"map_OFe_vs_FeH_included{config['FILE_SUFFIX']}.pdf")

    plt.close(fig)

In [ ]:
simulation = "au6_or_l4_s127"
plot_map_for_simulation(simulation=simulation,
                        abundances=[("Fe", 'H'), ('O', "Fe")],
                        xlim=(-1, 1), xticks=[-0.5, 0, 0.5],
                        ylim=(0.15, 0.35), yticks=[0.2, 0.25, 0.3])

In [33]:
sample = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]
plot_map_for_sample(sample=sample,
                    abundances=[("Fe", 'H'), ('O', "Fe")],
                    xlim=(-1, 1), xticks=[-0.5, 0, 0.5],
                    ylim=(0.15, 0.35), yticks=[0.2, 0.25, 0.3])